1. 패키지를 설치합니다.

In [ ]:
!pip install torch torchvision torchaudio accelerate transformers boto3 botocore

2. huggingpace 의 모델을 로딩하고 테스트/저장합니다..

In [ ]:
import torch
import boto3
from transformers import AutoModelForCausalLM, AutoTokenizer


device = "cuda"
model_path = "ibm-granite/granite-3.0-2b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)
# drop device_map if running on CPU
model = AutoModelForCausalLM.from_pretrained(model_path, device_map=device)
model.eval()
# change input text as desired
chat = [
    { "role": "user", "content": "Please list one IBM Research laboratory located in the United States. You should only output its name and location." },
]
chat = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
# tokenize the text
input_tokens = tokenizer(chat, return_tensors="pt").to(device)
# generate output tokens
output = model.generate(**input_tokens, 
                        max_new_tokens=100)
# decode output tokens into text
output = tokenizer.batch_decode(output)
# print output
print(output)

# 모델과 토크나이저 저장
save_directory = "./model/1"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)


2-1 (option) onnx 모델로 변환합니다.

In [ ]:
import torch
import boto3
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
model_path = "ibm-granite/granite-3.0-2b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 모델 및 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(model_path)
model.to(device)
model.eval()

# 예시 입력 데이터 (ONNX로 내보낼 때 사용할 입력 텐서)
chat = [
    { "role": "user", "content": "Please list one IBM Research laboratory located in the United States. You should only output its name and location." },
]
chat = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
input_tokens = tokenizer(chat, return_tensors="pt").to(device)

# ONNX로 변환
onnx_save_path = "./model/1/model.onnx"

# 모델을 ONNX로 내보내기
torch.onnx.export(
    model,                                  # PyTorch 모델
    args=(input_tokens['input_ids'],),      # 모델 입력 (input_ids를 사용)
    f=onnx_save_path,                       # ONNX 파일 경로
    input_names=["input_ids"],              # 입력 이름 정의
    output_names=["output"],                # 출력 이름 정의
    dynamic_axes={                          # 가변적인 입력 및 출력 크기 정의 (배치 크기)
        "input_ids": {0: "batch_size", 1: "sequence_length"}, 
        "output": {0: "batch_size", 1: "sequence_length"}
    },
    opset_version=14,                       # ONNX opset 버전 14로 수정
    do_constant_folding=True                # 상수 폴딩을 활성화하여 최적화
)

print(f"Model successfully exported to {onnx_save_path}")

# 토크나이저 저장
save_directory = "./model/1"
tokenizer.save_pretrained(save_directory)


3. data connection 으로 연결할 objectstorage 의 bucket 을 확인합니다.

In [4]:
#Upgrade pip to the latest version
!pip3 install --upgrade pip

#Install boto3
!pip3 install boto3

#Import boto3 libraries
import os 
import boto3 
from botocore.client import Config
from boto3 import session

#show Boto3 package version
!pip3 show boto3

#Create an S3 client

#Define credentials
key_id = os.environ.get('AWS_ACCESS_KEY_ID') 
secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY') 
endpoint = os.environ.get('AWS_S3_ENDPOINT')
region = os.environ.get('AWS_DEFAULT_REGION')

#Define client session
session = boto3.session.Session(aws_access_key_id=key_id, 
                                aws_secret_access_key=secret_key) 

#Define client connection
s3_client = boto3.client('s3', aws_access_key_id=key_id, 
                         aws_secret_access_key=secret_key,
                         aws_session_token=None,
                         config=boto3.session.Config(signature_version='s3v4'),
                         endpoint_url=endpoint,
                         region_name=region) 

#List available buckets
s3_client.list_buckets() 

#Print names of available buckets
for bucket in s3_client.list_buckets()['Buckets']: 
    print(bucket['Name'])

#List buckets - verify new bucket exists
for bucket in s3_client.list_buckets()['Buckets']: 
    print(bucket['Name'])

Name: boto3
Version: 1.34.162
Summary: The AWS SDK for Python
Home-page: https://github.com/boto/boto3
Author: Amazon Web Services
Author-email: 
License: Apache License 2.0
Location: /opt/app-root/lib/python3.9/site-packages
Requires: botocore, jmespath, s3transfer
Required-by: 
odyssey-data
odyssey-data


4. Bucket 에 업로드하고 파일을 확인합니다.

In [5]:
import os
import boto3
from botocore.client import Config

# Define credentials
key_id = os.environ.get('AWS_ACCESS_KEY_ID') 
secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY') 
endpoint = os.environ.get('AWS_S3_ENDPOINT')
region = os.environ.get('AWS_DEFAULT_REGION')
bucket_name = os.environ.get('AWS_S3_BUCKET')

# Define client connection
s3_client = boto3.client('s3', 
                         aws_access_key_id=key_id, 
                         aws_secret_access_key=secret_key,
                         config=Config(signature_version='s3v4'),
                         endpoint_url=endpoint,
                         region_name=region) 

# List available buckets
try:
    buckets = s3_client.list_buckets()
    # Print names of available buckets
    for bucket in buckets['Buckets']: 
        print(bucket['Name'])
except Exception as e:
    print(f"Error listing buckets: {e}")
    
#List files in a bucket
s3_client.list_objects_v2(Bucket=bucket_name) 

#Print only names of files
for bucket in s3_client.list_buckets()['Buckets']: 
    print(bucket['Name'])
    
directory = 'model/1'
s3_directory = 'model/1'
for filename in os.listdir(directory):
    local_path = os.path.join(directory, filename)

    if os.path.isfile(local_path):
        s3_path = os.path.join(s3_directory, filename)
        s3_client.upload_file(local_path, bucket_name, s3_path)

 #Print only names of files
for key in s3_client.list_objects_v2(Bucket=bucket_name)['Contents']:
    print(key['Key']) 

odyssey-data
odyssey-data
model/1/README.md
model/1/added_tokens.json
model/1/config.json
model/1/merges.txt
model/1/model-00001-of-00002.safetensors
model/1/model-00002-of-00002.safetensors
model/1/model.safetensors.index.json
model/1/special_tokens_map.json
model/1/tokenizer_config.json
model/1/vocab.json
